<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/RAG_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Complete Working Solution - No FAISS Required

# Install required packages (avoiding dependency conflicts)
!pip install langchain==0.3.27 \
            langchain-core==0.3.79 \
            langchain-community==0.3.31 \
            langchain-huggingface==0.3.1 \
            langchain-ollama==0.3.10 \
            sentence-transformers>=2.6.0

# Install faiss-cpu for vector operations
!pip install faiss-cpu

# Test imports
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# Initialize the embedding model (updated)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✅ All imports successful!")

# Simple test to see if embeddings work
test_embedding = embeddings.embed_query("Hello world")
print(f"✅ Embedding generated successfully: length {len(test_embedding)}")

# Test Ollama connection (you need ollama running)
try:
    chat_model = ChatOllama(model="llama3", temperature=0.2)
    print("✅ Ollama model initialized successfully")
except Exception as e:
    print(f"⚠️  Ollama not available: {e}")
    # Fallback to local model approach


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 93.8 MB/s eta 0:00:00
✅ All imports successful!
✅ Embedding generated successfully: length 384
✅ Ollama model initialized successfully


In [ ]:
# Simple RAG Demo - No External Dependencies Required

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import os

# Test embedding generation (this works)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("✅ Embedding model loaded successfully")

# Create a simple context-based approach without FAISS
def simple_context_retriever(query):
    """Simple function that returns relevant context based on query keywords"""

    # Sample knowledge base - in real app, this would be from your documents
    knowledge_base = {
        "python": "Python is a high-level programming language known for its simplicity and versatility.",
        "machine learning": "Machine learning is a subset of artificial intelligence that focuses on algorithms that improve automatically through experience.",
        "data science": "Data science combines statistics, computer science, and domain expertise to extract insights from data.",
        "jupyter": "Jupyter Lab is an interactive development environment for creating and sharing documents with live code."
    }

    # Simple keyword matching (in production you'd use vector similarity)
    matched_context = []
    query_lower = query.lower()

    for key, value in knowledge_base.items():
        if key in query_lower:
            matched_context.append(value)

    return "\n\n".join(matched_context) if matched_context else "I don't have specific information about that topic."

# Create prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant. Use the provided context to answer the question."),
    ("human", "Context: {context}\n\nQuestion: {question}")
])

def simple_rag_chain():
    """Create a minimal RAG chain without FAISS"""

    def rag_function(input_dict):
        question = input_dict["question"]
        context = simple_context_retriever(question)

        # Create the prompt
        messages = prompt_template.format_messages(context=context, question=question)
        return str(messages[0].content)  # Return just the content

    return RunnableLambda(rag_function)

# Example usage (this works without FAISS or Ollama)
chain = simple_rag_chain()

print("\n=== Testing Simple RAG Demo ===")
response1 = chain.invoke({"question": "What is Python?"})
print(f"Q: What is Python?\nA: {response1[:200]}...")

response2 = chain.invoke({"question": "machine learning algorithms"})
print(f"\nQ: machine learning algorithms\nA: {response2[:200]}...")


✅ Embedding model loaded successfully

=== Testing Simple RAG Demo ===
Q: What is Python?
A: You are an expert assistant. Use the provided context to answer the question....

Q: machine learning algorithms
A: You are an expert assistant. Use the provided context to answer the question....


In [ ]:
# Function to demonstrate RAG with real web data - Fixed Version
def demonstrate_rag_with_web_data():
    """Demonstrate RAG using web data"""

    # Install missing dependencies first
    try:
        from langchain_community.document_loaders import WebBaseLoader
        from langchain_community.vectorstores import FAISS
        from langchain_core.runnables import RunnablePassthrough, RunnableLambda
        from langchain_core.output_parsers import StrOutputParser
        from langchain_huggingface import HuggingFaceEmbeddings
        from langchain_core.prompts import ChatPromptTemplate
    except ImportError as e:
        print(f"Import error: {e}")
        return

    try:
        # Load a webpage
        url = "https://www.python.org/about/"
        loader = WebBaseLoader(web_path=url)
        docs = loader.load()

        print(f"Loaded {len(docs)} document(s) from {url}")

        # Create vector store from web data
        vector_store = FAISS.from_documents(docs, embeddings)

        # Create retriever
        retriever = vector_store.as_retriever(search_kwargs={"k": 3})

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        # Fixed: Proper chain construction with correct formatting
        rag_chain = (
            {"question": RunnablePassthrough(), "context": retriever}
            | RunnableLambda(lambda x: {"question": x["question"], "context": format_docs(x["context"])})
            | prompt_template
            | chat_model
            | StrOutputParser()
        )

        # Test questions
        test_questions = [
            "What is Python?",
            "What are some applications of machine learning?",
            "What is Jupyter Lab used for?",
            "What does the Python website say about its philosophy?"
        ]

        print("=== RAG with Web Data Demo ===")
        for i, question in enumerate(test_questions, 1):
            print(f"\nQuestion {i}: {question}")
            try:
                answer = rag_chain.invoke(question)
                print(f"Answer: {answer}")
            except Exception as e:
                print(f"Error: {e}")
                # Fallback to simple response
                print("Fallback: Could not generate answer with RAG")

    except Exception as e:
        print(f"Error in web data demo: {e}")
        print("This might be due to network issues or missing dependencies.")

        # Provide alternative approach for testing
        print("\n=== Fallback: Simple Context-Based Demo ===")
        simple_context_demo()

def simple_context_demo():
    """Simple demo without external dependencies"""
    context = """
    Python is a high-level programming language known for its simplicity and versatility.
    It was created by Guido van Rossum and first released in 1991.
    Python emphasizes code readability with significant use of indentation.

    Machine learning is a subset of artificial intelligence that enables computers to learn from data.
    Applications include image recognition, natural language processing, and predictive analytics.

    Jupyter Lab is an interactive development environment for creating and sharing documents
    containing live code, equations, visualizations, and narrative text.

    The Python philosophy emphasizes readability, simplicity, and ease of use.
    """

    # Simple question-answering without RAG
    questions = [
        "What is Python?",
        "What are some applications of machine learning?",
        "What is Jupyter Lab used for?",
        "What does the Python website say about its philosophy?"
    ]

    print("=== Simple Context-Based Demo ===")
    for i, question in enumerate(questions, 1):
        print(f"\nQuestion {i}: {question}")
        # Basic keyword matching
        answer = ""
        if "python" in question.lower():
            answer = "Python is a high-level programming language known for its simplicity and versatility."
        elif "machine learning" in question.lower():
            answer = "Machine learning applications include image recognition, natural language processing, and predictive analytics."
        elif "jupyter" in question.lower():
            answer = "Jupyter Lab is used for creating interactive documents with live code, visualizations, and narrative text."
        elif "philosophy" in question.lower() or "readability" in question.lower():
            answer = "Python's philosophy emphasizes readability, simplicity, and ease of use with significant indentation usage."

        if answer:
            print(f"Answer: {answer}")
        else:
            print("Answer: I don't have specific information about that topic.")

# Run the web data demo
try:
    demonstrate_rag_with_web_data()
except Exception as e:
    print(f"Final error: {e}")
    simple_context_demo()


Loaded 1 document(s) from https://www.python.org/about/
=== RAG with Web Data Demo ===

Question 1: What is Python?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 2: What are some applications of machine learning?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 3: What is Jupyter Lab used for?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 4: What does the Python website say about its philosophy?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG


In [ ]:
# Function to create a more advanced RAG chain with custom context - Fixed Version
def create_advanced_rag_chain():
    """Create an advanced RAG chain with custom context"""

    try:
        # Define custom context for demonstration
        custom_context = [
            "Python was created by Guido van Rossum and first released in 1991.",
            "Python is known for its readability and simple syntax, making it a great language for beginners.",
            "The Python Software Foundation supports the development of Python and its ecosystem.",
            "Python has a vast library ecosystem including NumPy, Pandas, TensorFlow, and PyTorch."
        ]

        # Create documents from custom context
        docs = [Document(page_content=context) for context in custom_context]

        # Create vector store
        vector_store = FAISS.from_documents(docs, embeddings)

        # Create retriever
        retriever = vector_store.as_retriever(search_kwargs={"k": 2})

        # Define a more complex prompt
        complex_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert Python developer. Use the provided context to answer technical questions about Python. Be detailed and accurate."),
            ("human", "Question: {question}"),
            ("placeholder", "{context}")
        ])

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        # Create the advanced RAG chain
        advanced_rag_chain = (
            {"question": RunnablePassthrough(), "context": retriever}
            | RunnableLambda(lambda x: {"question": x["question"], "context": format_docs(x["context"])})
            | complex_prompt
            | chat_model
            | StrOutputParser()
        )

        return advanced_rag_chain

    except Exception as e:
        print(f"Error creating advanced RAG chain: {e}")
        # Return a simple fallback chain if FAISS fails
        def simple_fallback_chain(question):
            context = "\n\n".join(custom_context)
            # Simple response without retrieval
            return f"Based on my knowledge, Python was created by Guido van Rossum in 1991. It's known for its readability and extensive library ecosystem including NumPy, Pandas, TensorFlow, and PyTorch."

        return simple_fallback_chain


In [ ]:
# Test the advanced RAG chain
def test_advanced_rag_chain():
    """Test the advanced RAG chain with sample questions"""

    try:
        # Create the advanced RAG chain
        advanced_chain = create_advanced_rag_chain()

        # Test questions
        test_questions = [
            "Who created Python and when was it first released?",
            "What makes Python good for beginners?",
            "What are some key libraries in the Python ecosystem?",
            "How does Python support machine learning development?"
        ]

        print("=== Testing Advanced RAG Chain ===")

        for i, question in enumerate(test_questions, 1):
            print(f"\nQuestion {i}: {question}")
            try:
                # Handle both callable types (chain vs function)
                if callable(advanced_chain):
                    answer = advanced_chain(question)
                    print(f"Answer: {answer}")
                else:
                    print("Chain not properly created")
            except Exception as e:
                print(f"Error processing question: {e}")

    except Exception as e:
        print(f"Error in test function: {e}")

# Run the test
test_advanced_rag_chain()


=== Testing Advanced RAG Chain ===

Question 1: Who created Python and when was it first released?
Chain not properly created

Question 2: What makes Python good for beginners?
Chain not properly created

Question 3: What are some key libraries in the Python ecosystem?
Chain not properly created

Question 4: How does Python support machine learning development?
Chain not properly created


In [ ]:
# RAG (Retrieval-Augmented Generation) Demo - Fixed Version

# Main execution
if __name__ == "__main__":
    print("=== RAG (Retrieval-Augmented Generation) Demo ===\n")

    # Create and test basic RAG chain
    print("1. Basic RAG Chain with Sample Documents")
    basic_rag = create_rag_chain()

    test_questions = [
        "What is Python?",
        "What is machine learning?",
        "What is data science?",
        "What is Jupyter Lab?"
    ]

    for question in test_questions:
        try:
            answer = basic_rag.invoke(question)
            print(f"Question: {question}")
            print(f"Answer: {answer}\n")
        except Exception as e:
            print(f"Error: {e}\n")

    # Demonstrate RAG with web data
    print("2. RAG with Web Data")
    demonstrate_rag_with_web_data()

    # Create and test advanced RAG chain
    print("\n3. Advanced RAG Chain with Custom Context")
    advanced_rag = create_advanced_rag_chain()

    advanced_test_questions = [
        "Who created Python?",
        "Why is Python popular among beginners?",
        "What are some major Python libraries?",
        "What is the Python Software Foundation?"
    ]

    print("\nAdvanced RAG Answers:")
    for question in advanced_test_questions:
        try:
            answer = advanced_rag.invoke(question)
            print(f"Question: {question}")
            print(f"Answer: {answer}\n")
        except Exception as e:
            print(f"Error: {e}\n")

    print("\n=== RAG Demo Complete ===")
    print("You can modify the documents or prompts to test different scenarios.")
    print("For real-world applications, you would typically use a database or API to fetch context.")


=== RAG (Retrieval-Augmented Generation) Demo ===

1. Basic RAG Chain with Sample Documents
Error: [Errno 111] Connection refused

Error: [Errno 111] Connection refused

Error: [Errno 111] Connection refused

Error: [Errno 111] Connection refused

2. RAG with Web Data
Loaded 1 document(s) from https://www.python.org/about/
=== RAG with Web Data Demo ===

Question 1: What is Python?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 2: What are some applications of machine learning?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 3: What is Jupyter Lab used for?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

Question 4: What does the Python website say about its philosophy?
Error: [Errno 111] Connection refused
Fallback: Could not generate answer with RAG

3. Advanced RAG Chain with Custom Context

Advanced RAG Answers:
Error: variable context should be a list of b